In [3]:
import visa
import time
import numpy as np

In [4]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR', 'ASRL10::INSTR', 'GPIB0::14::INSTR')

In [5]:
ametek7270 = rm.open_resource('USB0::0x0A2D::0x001B::09140123::RAW')
keithley6221 = rm.open_resource('GPIB0::14::INSTR')
gauss = rm.open_resource('GPIB0::14::INSTR')

In [6]:
SENSITIVITIES = [
        0.0, 2.0e-9, 5.0e-9, 10.0e-9, 20.0e-9, 50.0e-9, 100.0e-9,
        200.0e-9, 500.0e-9, 1.0e-6, 2.0e-6, 5.0e-6, 10.0e-6,
        20.0e-6, 50.0e-6, 100.0e-6, 200.0e-6, 500.0e-6, 1.0e-3,
        2.0e-3, 5.0e-3, 10.0e-3, 20.0e-3, 50.0e-3, 100.0e-3,
        200.0e-3, 500.0e-3, 1.0
    ]
TIME_CONSTANTS = [
        10.0e-6, 20.0e-6, 50.0e-6, 100.0e-6, 200.0e-6, 500.0e-6,
        1.0e-3, 2.0e-3, 5.0e-3, 10.0e-3, 20.0e-3, 50.0e-3, 100.0e-3,
        200.0e-3, 500.0e-3, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0,
        100.0, 200.0, 500.0, 1.0e3, 2.0e3, 5.0e3, 10.0e3,
        20.0e3, 50.0e3, 100.0e3
    ]

In [62]:
def init_ac(ac, freq=157, start_I=0.002, stand_by_I=0.002):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:AMPL {}".format(start_I))
    ac.write("SOUR:WAVE:FREQ {}".format(freq))
    ac.write("SOUR:WAVE:ARM")
    ac.write("SOUR:WAVE:INIT")
    time.sleep(3)
    ac.write("SOUR:WAVE:AMPL {}".format(stand_by_I))
    

In [10]:
def init_ametek(ame, time_constant=0.5, sensitivity=10.0e-6, harmonic=1):
    ame.write("TC {}".format(TIME_CONSTANTS.index(time_constant)))
    ame.write("SEN {}".format(SENSITIVITIES.index(sensitivity)))
    ame.write("REFN {}".format(harmonic))
    

In [11]:
def lockin_measurement(ame, keith, ac_A=0.001, standby_I=0.0005,  duration=4, time_per_point=0.1, points_stored=5, show_data=False):
    X = []
    Y = []
    M = []
    P = []
    keith.write("SOUR:WAVE:AMPL {}".format(ac_A))
    for i in range(int(duration/time_per_point)):
        time.sleep(time_per_point)
        m, p = ame.query('MP.').replace("\n\x00", "").split(",")
        x, y = ame.query('XY.').replace("\n\x00", "").split(",")
        X.append(float(x))
        Y.append(float(y))
        M.append(float(m))
        P.append(float(p))
    keith.write("SOUR:WAVE:AMPL {}".format(standby_I))
    res = []
    all_result = [X, Y, M, P]
    for i in all_result:
        i = i[-points_stored:]
        i = np.array(i)
        res.append(i.mean())
        res.append(i.std())
    if show_data:
        print(all_result)
    return res

In [35]:
init_ametek(ametek7270, time_constant=0.1, sensitivity=0.00005, harmonic=2)

In [66]:
init_ac(keithley6221, freq=163, stand_by_I=0.0005)

do first harmonic measurement, get lowest AC GAIN input limit,(low ac gain input limit, low backgraound noise for 2nd harmnic result) auto phase,

In [65]:
lockin_measurement(ametek7270, keithley6221, ac_A=0.006, standby_I=0.0005,  duration=3, time_per_point=0.1,
                   points_stored=10, show_data=True)

[[-0.009455, -0.00645, 0.005025, 0.02047, 0.017175, -0.074395, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.149625, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.001005], [-0.042835, -0.05161, -0.090665, -0.15, -0.15, -0.15, -0.05495, 0.056335, 0.124835, 0.135045, 0.126055, 0.132825, 0.138365, 0.118765, 0.0821, 0.04436, 0.016365, -0.006885, -0.04297, -0.09033, -0.12666, -0.134015, -0.078755, 0.076325, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15], [0.043845, 0.05201, 0.0908, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15], [-102.47, -97.12, -86.83, -82.23, -83.47, -99.28, -159.88, 159.42, 140.23, 138.0, 139.96, 138.48, 137.3, 141.63, 151.31, 163.53, 173.78, -177.37, -164.02, -148.95, -139.82, -138.15, -151.72, 153.03, 135.0, 135.0, 135.0, 135.0, 135.0, 90.96]]


[-0.135063,
 0.04468613986461573,
 0.0636895,
 0.11851882018164878,
 0.14999999999999997,
 2.7755575615628914e-17,
 48.92999999999999,
 126.70022367778205]

In [232]:
Pos = [1,1,2,2,3,3,3,3,3,3,3,3,3,4,4,4,4,5,6,7] + [8]*20
Pos_index = [0]

In [233]:
def get_position():
    Pos_index[0] += 1
    return Pos[Pos_index[0]-1]

In [38]:
import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

In [29]:
x = get_position()

In [27]:
round(1.9)

2

In [30]:
print(x)

2


In [39]:
def angular_measurement(ame, keith, sample_id="CoFe_Pt_RXX_2nd", ac_A=0.005, standby_I=0.0005,  duration=3, 
                        time_per_point=0.1, points_stored=10, counts_per_angle=4, wait_time_out=10,
                        wait_per_measurement=5, freq=153):
    filename = sample_id + "_ac_{}_duration_{}_f_{}.csv".format(ac_A, duration, freq)
    with open(filename, 'w') as f:
        f.write("X,dX,Y,dY,M,dM,P,dP,POS\n")
    point_count = 0
    count = 0 # if position didn't change count++, stop measurement once count > 
    last_position = None
    while count < wait_time_out:
        pos = get_position()
        print("current position: {}".format(pos))
        if pos == last_position and point_count > counts_per_angle:
            count += 1
            time.sleep(0)
        else:
            res = lockin_measurement(ame, keith, ac_A=ac_A, standby_I=standby_I,  duration=duration,
                               time_per_point=time_per_point, points_stored=points_stored)
            res.append(pos)
            res = [str(i) for i in res]
            with open(filename, 'a') as f:
                f.write(",".join(res) + "\n")
            time.sleep(wait_per_measurement)
            if pos == last_position:
                point_count += 1
            else:
                point_count =0
            last_position = pos
            count = 0
                

In [41]:
do_measurment = True

In [58]:
if not do_measurment:
    raise
angular_measurement(ametek7270, keithley6221, sample_id="CoFe_SIO_RXy_2nd_test_scan", ac_A=0.006, standby_I=0.0005,  duration=3, 
                        time_per_point=0.1, points_stored=5, counts_per_angle=20, wait_time_out=80,
                        wait_per_measurement=10, freq=163)
do_measurment=False

current position: 0
current position: -3
current position: 0
current position: 0
current position: 0
current position: 0
current position: 0
current position: 20
current position: 20
current position: 20
current position: 20
current position: 32
current position: 40
current position: 40
current position: 40
current position: 40
current position: 60
current position: 60
current position: 60
current position: 60
current position: 60
current position: 80
current position: 80
current position: 80
current position: 80
current position: 80
current position: 100
current position: 100
current position: 100
current position: 100
current position: 111


PermissionError: [Errno 13] Permission denied: 'CoFe_SIO_RXx_2nd_test_scan_ac_0.0008_duration_3_f_163.csv'